### Ingest circuits.csv file

In [ ]:
dbutils.widgets.text("p_data_source", "")
v_data_source = dbutils.widgets.get("p_data_source")

dbutils.widgets.text("p_file_date", "2021-03-21")
v_file_date = dbutils.widgets.get("p_file_date")

In [ ]:
from formula1.formula1_constants import *
from formula1.formula1_utils import *

##### Step 1 - Read the CSV file using the spark dataframe reader

In [ ]:
from pyspark.sql.types import (
    StructType,
    StructField,
    IntegerType,
    StringType,
    DoubleType,
)
from pyspark.sql.functions import col, lit

circles_schema = StructType(
    fields=[
        StructField("circuitId", IntegerType(), False),
        StructField("circuitRef", StringType(), True),
        StructField("name", StringType(), True),
        StructField("location", StringType(), True),
        StructField("country", StringType(), True),
        StructField("lat", DoubleType(), True),
        StructField("lng", DoubleType(), True),
        StructField("alt", IntegerType(), True),
        StructField("url", StringType(), True),
    ]
)

# Read from Landing Zone using date partition
circuits_df = (
    spark.read.option("header", True)
    .schema(circles_schema)
    .csv(f"{landing_folder_path}/{v_file_date}/circuits.csv")
)

##### Step 2 - Add ingestion meta data

In [ ]:
circuits_final_df = (
    add_ingestion_date(circuits_df)
    .withColumn("data_source", lit(v_data_source))
    .withColumn("file_date", lit(v_file_date))
)

##### Step 3 - Write data to bronze container

In [ ]:
circuits_final_df.write.mode("overwrite").format("delta").saveAsTable(
    "f1_bronze.circuits"
)